## Rubric
|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Zhengyin Yang :Revise Ethics and team expectations. Coding / Use tools to visualize the data cleanliness.

Yanbai Li : Obtain more data and do the brief wragling and pre-clean the datas for the two datasets.

Alisa Yang : Revise with hypothesis. DataSet1 description.

Lily Cheng : Dataset2 desription. Double check with dataset1 and 2

Keyuan Wang: .

## Research Question

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

From 2018,2020,2022,2024 at UC San Diego (UCSD), how do transfer-admit selectivity patterns differ across ten selected majors, and how are these patterns associated with major-level alumni earnings? Using major × year aggregated data, we will characterize selectivity using admitted GPA ranges (converted to midpoint and spread) and admission rates, and we will estimate year-to-year trends for each major. We will then merge in major-level earnings outcomes (25th/median/75th percentile annual earnings and earner counts) for the corresponding year and test whether majors with higher earnings tend to have higher admitted GPA midpoints and/or lower admission rates. Our analysis will use descriptive summaries, trend comparisons, and correlation/OLS regression models (with controls for year, major fixed effects, and applicant volume where available) to assess the strength and significance of these relationships. The final deliverables will identify (1) which majors appear most selective in each year, (2) which majors show the fastest changes in selectivity from 2018,2020,2022,2024, and (3) whether major-level earnings are statistically associated with selectivity measures in this period.

## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

Choosing a university major is a crucial decision that impacts a student's academic trajectory and future career prospects. In recent years, there has been increasing attention to the "return on investment" of higher education, with prospective students often weighing the academic difficulty of a major against its potential graduate earnings. For transfer students—who typically have more defined career goals upon entering university—the competitiveness of admission (measured by the GPA distribution of admitted students (25th, 50th, and 75th percentiles)) is a primary evaluation criterion. This study uses the University of California, San Diego (UCSD) as a case study, a leading public research university where transfer admission selectivity and GPA thresholds vary significantly across different disciplines.

Previous research has confirmed a significant but nuanced link between university admission difficulty and post-graduation earnings. Data from the University of California Information Center suggests this relationship is particularly pronounced at highly competitive institutions like UCSD, especially in fields such as mechanical engineering and computer science, where UC alumni consistently rank among the highest earners five years after graduation. On the other hand, some "pre-professional" majors, such as human biology, maintain high GPA admission thresholds despite potentially lower undergraduate earnings compared to technical fields. This is often because these students go on to pursue advanced medical degrees, indicating that "admission difficulty" is not solely determined by current average salaries but also by long-term academic value.

Labor market conditions and alumni salary signals have also been shown to influence student demand, and consequently, the academic standards required for admission. It is widely believed that students tend to gravitate towards majors perceived to have higher economic returns, such as STEM fields and applied social sciences like business psychology. This shift in demand leads to increased admission difficulty and higher GPA admission thresholds. For example, UCSD's computer science program consistently maintains higher GPA requirements than humanities disciplines like history, directly reflecting its leading position in alumni earnings data.

Despite these observations, there is a lack of localized research specifically examining the year-to-year changes in UCSD's admission GPA distribution and its correlation with real-time average salary trends for specific majors. While general research suggests that a 10% increase in expected salary increases the likelihood of students choosing a particular major by nearly 14% to 18%, it remains unclear whether these enrollment preferences immediately translate into quantifiable changes in the 25th, and 75th percentiles of matriculation GPA. This project aims to bridge the gap between educational enrollment data and labor market wage signals by analyzing fifteen different majors at the University of California, San Diego (UCSD), spanning fields from history to mechanical engineering.

## Hypothesis


We hypothesize that higher post-graduation salaries lead to a larger, more competitive applicant pool, causing a measurable increase in admitted transfer GPAs for that major. We expect to see a positive correlation between changes in major-level median salary and changes in GPA selectivity across the observed period.

Students tend to choose majors with better employment prospects. However, there is a time lag between observing the job market and actually applying. Students applying for the fall 2024 admission are likely to make their decisions based on the salary levels of the 2021-2022 graduates. At UCSD, many majors have enrollment limit. Therefore, the increase in the number of applicants influenced by the previous high income should lead to an increase in the admission GPA threshold in the coming years. We expect this impact to be most significant in computer science and engineering majors.

How we will conduct the test: We will calculate the percentage changes in median salary and the percentile changes in admission GPA. We will conduct a correlation test to determine if the GPA increases more rapidly in professional fields with greater prior salary growth. We will also group the majors by fields (STEM, humanities) and conduct separate regression analyses to examine if there are differences in the relationships between different fields.



## Data
An ideal dataset would contain individual transfer applicant records from UCSD, each applicant's major, transfer GPA, admission decision, and entry year. This would let us calculate any GPA percentile we need rather than relying on pre-aggregated summaries. These applicant records would be linked to actual wage data. The earnings data would be UCSD-specific. We need four years of data (2018,2020,2022,2024) across fifteen to twenty majors to see real trends, about 100 to 150 total observations. The data would live in a secure database with student privacy protected.

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:2018-2024 transfer studnets' major admission
  - Link to the dataset:https://github.com/COGS108/Group084_WI26/blob/master/data/00-raw/2018-2024%20Major%20Table.csv
  - Number of observations:60 (15 majors × 4 years)
  - Number of variables:9 (Year, Major, Applicants, Admits, Enrollees, Admit GPA range, Enrollee GPA range, Admit rate, Yield rate)
  - Description of the variables most relevant to this project: Admit GPA range: The most crucial variable, expressed as the middle 50% of GPAs for admitted students (e.g., "3.41 – 3.96"). The lower limit (25th percentile) serves as the primary dependent variable, representing the selectivity threshold that is hypothesized to rise in response to high salary outcomes; Applicants: The total number of transfer students who applied to a specific major, serving as the main indicator for measuring project demand; Major: Categorizes the data into specific fields (e.g., humanities, social sciences, life sciences, engineering), allowing for comparison across disciplines; Year: Allows for tracking changes in selectivity and demand over the four time points (2018, 2020, 2022, 2024).
  - Descriptions of any shortcomings this dataset has with repsect to the project: Aggregated data: The data is grouped by major, preventing analysis of individual student-level factors such as their community college of origin or geographic location; GPA ceiling: Because GPA is capped at 4.0, competitive majors often have an upper bound of 4.00. This creates a hard limit, making it impossible to measure further improvements in the strength of top-tier students; Small sample sizes for some majors: Majors with very few enrolling students are susceptible to significant volatility in their GPA averages, as they can be heavily influenced by the data of just one or two individuals. 






- Dataset #2
  - Dataset Name:UC undergrad alumni annual earnings after two years of gradutaion from 2015-2021.
  - Link to the dataset:https://github.com/COGS108/Group084_WI26/blob/master/data/00-raw/UC%20alumni%20at%20work.csv
  - Number of observations:60(15 majors × 4 year groups)
  - Number of variables:4(75%, 50%, 25% of annual earnings; Count of Number of Records) 
  - Description of the variables most relevant to this project: 
     The top tier(75%) low tier(25%) and average (50% / median) annual earning for a typical University of California alumni whom graduated in 2015-2016, 2017-2018, 2019-2022, and 2022-2023, aggregated by major. Note that the annual salaries here have been adjusted to 2024 dollar, and represents the average annual salaries UC bachelor degree owners earned within 2 years of graduation.

     Count of Number of Records. Is the numbers of UC alumni that give permission for UC to record and use their data anonymously.

  - Descriptions of any shortcomings this dataset has with repsect to the project: 
     This data is based on the Count of Number of Records, which many whom didn’t earn quite a good living would tend to not be on the record. Though we try our best to choose the represent majors, some rows have very few of UC alumni being recorded, for example, there is only 30 people are recorded in chemistry in 2015-2016 so their salary averages can swing wildly. Also, as for UC San Diego alumni only, the numbers of recorded are way too low beyond measurable, thus hereby we choose the UC wide alumni as representing UCSD alumni, which different schools may have different recognized majors, and this ego effect might also affect the annual salaries one may earned. For example, UCLA’s Visual art -media major is world-wide recognized, thus for when we compared UCSD’s enrollment (the first and previous dataset here) with UC alumni salaries (the second and this dataset here), we can’t reject there is a possibility here that the earning we used here is bit higher than the actual UCSD media major earning.









In [1]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [18]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    { 'url': 'https://raw.githubusercontent.com/COGS108/Group084_WI26/cef6fe107a44418098ad1f67f63c8df132ba643f/data/00-raw/2018-2024%20Major%20Table.csv', 'filename':'2018-2024 Major Table.csv'},
    { 'url': 'https://raw.githubusercontent.com/COGS108/Group084_WI26/cef6fe107a44418098ad1f67f63c8df132ba643f/data/00-raw/UC%20alumni%20at%20work.csv', 'filename':'UC alumni at work.csv'}
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')

Overall Download Progress:  50%|█████     | 1/2 [00:00<00:00,  6.86it/s]          

Successfully downloaded: 2018-2024 Major Table.csv



Overall Download Progress: 100%|██████████| 2/2 [00:00<00:00,  6.87it/s]      

Successfully downloaded: UC alumni at work.csv


### Major Dataset: UC Major Selectivity & GPA (2018–2024)

Instructions: 
1. Change the header from Dataset #1 to something more descriptive of the dataset
2. Write a few paragraphs about this dataset. Make sure to cover
   1. Describe the important metrics, what units they are in, and giv some sense of what they mean.  For example "Fasting blood glucose in units of mg glucose per deciliter of blood.  Normal values for healthy individuals range from 70 to 100 mg/dL.  Values 100-125 are prediabetic and values >125mg/dL indicate diabetes. Values <70 indicate hypoglycemia. Fasting idicates the patient hasn't eaten in the last 8 hours.  If blood glucose is >250 or <50 at any time (regardless of the time of last meal) the patient's life may be in immediate danger"
   2. If there are any major concerns with the dataset, describe them. For example "Dataset is composed of people who are serious enough about eating healthy that they voluntarily downloaded an app dedicated to tracking their eating patterns. This sample is likely biased because of that self-selection. These people own smartphones and may be healthier and may have more disposable income than the average person.  Those who voluntarily log conscientiously and for long amounts of time are also likely even more interested in health than those who download the app and only log a bit before getting tired of it"


This dataset contains the admission statistics of transfer students from UCSD for the years 2018, 2020, 2022, and 2024, aggregated by major. The data was extracted from the university's institutional research records and provides a view of the selective trends in admission choices across 15 different majors in the fields of humanities, social sciences, life sciences, and engineering. The data structure consists of each row representing a unique combination of major and year, and all fields have been cleaned and validated for consistency.

The column of applicants represents the total number of transfer students who submitted applications to the specific major during the admission period, and it is the main indicator for measuring the project's demand. Admissions indicate the number of applicants who were admitted. The admission rate is calculated by dividing the number of admissions by the number of applicants, reflecting the competitiveness from the perspective of the institution's enrollment capacity. The enrollment number counts the number of admitted students who eventually enrolled at the University of California, San Diego. The enrollment rate is calculated by dividing the enrollment number by the number of admissions, reflecting the attractiveness of the admission to this major compared to other institutions.

Our assumption holds that the most crucial indicator is the GPA range. The admission GPA range is expressed by the lowest and highest values (for example, "3.41 – 3.96"), which actually represent the middle 50% of the transfer GPAs of admitted students. The lower limit corresponds to the 25th percentile, indicating the GPA required to be a competitive candidate, while the upper limit corresponds to the 75th percentile, showing the GPA of high-performing admitted students. The admission GPA range follows the same structure but reflects the actual admitted students, excluding those who have been admitted to other institutions. 

This dataset is directly relevant to our hypothesis regarding salary-driven competition. The Admit GPA range, specifically the 25th and 75th percentiles, serves as our primary dependent variable, representing the selectivity threshold we hypothesize will rise in response to strong labor market signals. An increase in applicants in specific major driven by high salary outcomes should force admissions officers to raise the GPA floor. We can observe this by tracking changes in the lower bound of the Admit GPA range over time.

Several limitations of this dataset should be noted. The data is grouped by major, so we cannot see individual student information like which community college they came from or where they live. Since GPA cannot go above 4.0, competitive majors like mechanical engineering often has upper bound 4.00, so if top students get even stronger, we cannot measure that improvement because the scale has a hard limit. Some majors have very few enrolling students, so their GPA averages can swing wildly based on just one or two people. Also, we assume that every person here is rational, who makes decisions and holds beliefs based on logic, objective facts, and evidence rather than on emotions or impulses. But in reality, we have to admit that people aren't always rational and that may affect our when we try to conclude on whether .  


3. Use the cell below to 
    1. load the dataset 
    2. make the dataset tidy or demonstrate that it was already tidy
    3. demonstrate the size of the dataset
    4. find out how much data is missing, where its missing, and if its missing at random or seems to have any systematic relationships in its missingness
    5. find and flag any outliers or suspicious entries
    6. clean the data or demonstrate that it was already clean.  You may choose how to deal with missingness (dropna of fillna... how='any' or 'all') and you should justify your choice in some way
    7. You will load raw data from `data/00-raw/`, you will (optionally) write intermediate stages of your work to `data/01-interim` and you will write the final fully wrangled version of your data to `data/02-processed`
4. Optionally you can also show some summary statistics for variables that you think are important to the project
5. Feel free to add more cells here if that's helpful for you


#### Outlier & Suspicious Entry Flags

In [23]:
df_major_flag = df_major.copy()
rules = []
rules.append(("bad_year", ~df_major_flag["year"].isin([2018, 2020, 2022, 2024])))
if "admit_rate" in df_major_flag.columns:
    rules.append(("bad_admit_rate", (df_major_flag["admit_rate"] < 0) | (df_major_flag["admit_rate"] > 1)))
if "yield_rate" in df_major_flag.columns:
    rules.append(("bad_yield_rate", (df_major_flag["yield_rate"] < 0) | (df_major_flag["yield_rate"] > 1)))

for c in ["applicants", "admits", "enrollees"]:
    if c in df_major_flag.columns:
        rules.append((f"neg_{c}", df_major_flag[c] < 0))

if set(["applicants","admits"]).issubset(df_major_flag.columns):
    rules.append(("admits_gt_applicants", df_major_flag["admits"] > df_major_flag["applicants"]))
if set(["enrollees","admits"]).issubset(df_major_flag.columns):
    rules.append(("enrollees_gt_admits", df_major_flag["enrollees"] > df_major_flag["admits"]))
if "gpa_mid" in df_major_flag.columns:
    rules.append(("bad_gpa_mid", (df_major_flag["gpa_mid"] < 0) | (df_major_flag["gpa_mid"] > 4.5)))

def flag_iqr_outliers(s, k=1.5):
    s = s.dropna()
    if len(s) < 6:
        return pd.Series(False, index=s.index) 
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - k * iqr, q3 + k * iqr
    return (s < lo) | (s > hi)

iqr_cols = [c for c in ["applicants","admits","enrollees","gpa_mid","admit_rate","yield_rate"] if c in df_major_flag.columns]
for col in iqr_cols:
    mask = (
        df_major_flag.groupby("year", group_keys=False)[col]
        .apply(lambda s: flag_iqr_outliers(s, k=1.5))
    )
    df_major_flag[f"out_{col}_iqr"] = False
    df_major_flag.loc[mask.index, f"out_{col}_iqr"] = mask

df_major_flag["suspicious_reasons"] = ""
for name, cond in rules:
    df_major_flag.loc[cond.fillna(False), "suspicious_reasons"] += f"{name};"

for col in iqr_cols:
    flag_col = f"out_{col}_iqr"
    df_major_flag.loc[df_major_flag[flag_col] == True, "suspicious_reasons"] += f"{flag_col};"

df_major_flag["is_suspicious"] = df_major_flag["suspicious_reasons"].str.len() > 0
print("Suspicious rows (major):", df_major_flag["is_suspicious"].sum())
display(df_major_flag[df_major_flag["is_suspicious"]].head(30))

Suspicious rows (major): 9


,year,major,applicants,admits,enrollees,admit_rate,yield_rate,out_applicants_iqr,out_admits_iqr,out_enrollees_iqr,out_admit_rate_iqr,out_yield_rate_iqr,suspicious_reasons,is_suspicious
6,2024,Cognitive Science,358,294,140,0.82,0.48,False,True,True,False,False,out_admits_iqr;out_enrollees_iqr;,True
21,2022,Cognitive Science,313,257,107,0.82,0.42,False,False,True,False,False,out_enrollees_iqr;,True
36,2020,Cognitive Science,278,195,86,0.70,0.44,False,False,True,False,False,out_enrollees_iqr;,True
44,2020,Mechanical Engineering,540,149,40,0.28,0.27,True,False,False,False,False,out_applicants_iqr;,True
49,2018,Electrical Engineering - BS,435,221,69,0.51,0.31,True,False,True,False,False,out_applicants_iqr;out_enrollees_iqr;,True
51,2018,Cognitive Science,254,173,73,0.68,0.42,False,False,True,False,False,out_enrollees_iqr;,True
54,2018,Literature/Writing,101,57,14,0.56,0.25,False,False,True,False,False,out_enrollees_iqr;,True
58,2018,"Ecology, Behavior & Evolution",66,22,4,0.33,0.18,False,False,True,False,False,out_enrollees_iqr;,True
59,2018,Mechanical Engineering,560,137,76,0.24,0.55,True,False,True,False,False,out_applicants_iqr;out_enrollees_iqr;,True


#### Tidy and Clean Data

In [25]:
import os, re
import numpy as np
import pandas as pd

# ---- paths (repo template folders) ----
RAW_DIR = "data/00-raw"
PROC_DIR = "data/02-processed"
os.makedirs(PROC_DIR, exist_ok=True)

major_path = os.path.join(RAW_DIR, "2018-2024 Major Table.csv")

df_major_raw = pd.read_csv(
    major_path,
    encoding="utf-8-sig",
    sep=",",
    engine="python",
)

print("Major raw shape:", df_major_raw.shape)
display(df_major_raw.head())

def to_numeric_percent(x):
    """'75%' -> 0.75, '0.75'->0.75, NaN stays NaN"""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "":
        return np.nan
    s = s.replace("%", "")
    try:
        v = float(s)
    except:
        return np.nan
    # if user stored 75 instead of 0.75
    if v > 1.0:
        v = v / 100.0
    return v

def parse_gpa_range(x):
    """'3.41 - 3.96' -> (3.41, 3.96); returns (nan,nan) if fails"""
    if pd.isna(x):
        return (np.nan, np.nan)
    s = str(x)
    # normalize dash variants
    s = s.replace("–", "-").replace("—", "-")
    # extract two numbers
    nums = re.findall(r"\d+\.\d+|\d+", s)
    if len(nums) >= 2:
        lo = float(nums[0]); hi = float(nums[1])
        if lo > hi:
            lo, hi = hi, lo
        return (lo, hi)
    return (np.nan, np.nan)

def standardize_major_name(x):
    """basic cleanup; keep it conservative"""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s
rename_map = {
    "Year": "year",
    "Major": "major",
    "Applicants": "applicants",
    "Admits": "admits",
    "Enrollees": "enrollees",
    "Admit GPA": "admit_gpa_range",
    "Enrollee GPA": "enrollee_gpa_range",
    "Admit rate": "admit_rate",
    "Yield rate": "yield_rate",
}
df_major = df_major_raw.rename(columns=rename_map).copy()

keep_cols = [c for c in rename_map.values() if c in df_major.columns]
df_major = df_major[keep_cols].copy()
df_major["year"] = pd.to_numeric(df_major["year"], errors="coerce").astype("Int64")
df_major["major"] = df_major["major"].apply(standardize_major_name)
lo, hi = zip(*df_major["admit_gpa_range"].apply(parse_gpa_range)) if "admit_gpa_range" in df_major.columns else ([], [])
if len(lo) > 0:
    df_major["gpa_low"] = lo
    df_major["gpa_high"] = hi
    df_major["gpa_mid"] = (df_major["gpa_low"] + df_major["gpa_high"]) / 2
    df_major["gpa_width"] = df_major["gpa_high"] - df_major["gpa_low"]

if "admit_rate" in df_major.columns:
    df_major["admit_rate"] = df_major["admit_rate"].apply(to_numeric_percent)
if "yield_rate" in df_major.columns:
    df_major["yield_rate"] = df_major["yield_rate"].apply(to_numeric_percent)

for c in ["applicants", "admits", "enrollees"]:
    if c in df_major.columns:
        df_major[c] = pd.to_numeric(df_major[c], errors="coerce")

YEARS = [2018, 2020, 2022, 2024]
df_major = df_major[df_major["year"].isin(YEARS)].copy()

df_major = df_major.dropna(subset=["year", "major"]).copy()

print("Major cleaned shape:", df_major.shape)
display(df_major.head(10))

miss = df_major.isna().mean().sort_values(ascending=False)
print("Major missingness (fraction):")
display(miss)

major_out = os.path.join(PROC_DIR, "major_processed.csv")
df_major.to_csv(major_out, index=False, encoding="utf-8-sig")
print("Saved:", major_out)

Major raw shape: (60, 9)


,Year,Major,Applicants,Admits,Enrollees,Admit GPA range,Enrollee GPA range,Admit rate,Yield rate
0,2024,History,284,212,24,3.41 - 3.96,3.16 - 3.72,75%,11%
1,2024,Visual Arts - Media,263,176,63,3.41 - 3.88,3.44 - 3.82,67%,36%
2,2024,Computer Engineering,455,137,61,3.70 - 3.97,3.72 - 3.93,30%,45%
3,2024,Chemistry,190,89,28,3.41 - 3.98,3.40 - 3.69,47%,31%
4,2024,Electrical Engineering - BS,412,161,68,3.51 - 3.95,3.40 - 3.82,39%,42%


Major cleaned shape: (60, 7)


,year,major,applicants,admits,enrollees,admit_rate,yield_rate
0,2024,History,284,212,24,0.75,0.11
1,2024,Visual Arts - Media,263,176,63,0.67,0.36
2,2024,Computer Engineering,455,137,61,0.30,0.45
3,2024,Chemistry,190,89,28,0.47,0.31
4,2024,Electrical Engineering - BS,412,161,68,0.39,0.42
5,2024,Bioengineering,95,41,14,0.43,0.34
6,2024,Cognitive Science,358,294,140,0.82,0.48
7,2024,Public Health,217,128,56,0.59,0.44
8,2024,Applied Mathematics,194,143,46,0.74,0.32
9,2024,Literature/Writing,153,90,13,0.59,0.14


Major missingness (fraction):


year          0.0
major         0.0
applicants    0.0
admits        0.0
enrollees     0.0
admit_rate    0.0
yield_rate    0.0
dtype: float64

Saved: data/02-processed/major_processed.csv


### Earnings Dataset: UC Alumni Earnings by Major (2018–2024) 

This dataset contains the admission statistics of annual earning for a typical University of California alumni whom graduated in 2015-2016, 2017-2018, 2019-2022, and 2022-2023, aggregated by major. The data was extracted from the UC alumni at work, information center website, who collects its’ data from AEO, financial aid and degree data, the USDOE's College Scorecard, National Association of Colleges and Employers (NACE) Survey 2015 and MIT Living wage calculator.Note that the annual salaries here have been adjusted to 2024 dollar, and represents the average annual salaries UC bachelor degree owners earned within 2 years of graduation.

This dataset has 15 different majors across the wide fields of humanities, social sciences, life sciences, and engineering. The rows represent a unique combination of major and year, and all fields have been cleaned and validated for consistency. The column of earnings percentile 75, median, and percentile 25, gives us an overall view of how the annual earnings would look like for a student who earned a bachelor in a specific major, didn’t go to any graduate school and work directly for 2 years. Count of Number of Records is the number of UC alumni that are recorded. Which quite matches the reality of engineering students tend and more easily to get a job right after undergrad, while natural science and art degrees are more likely pursuing a higher degree. We also try using the rate of records, which means the count of number of records divided by the overall graduated where we can’t find the specific data of UC alumni numbers in a specific major (only departments’ count).

So, 1. for a certain major; (when the median doesn't have a significant change), if the 75% earning goes higher or the 25% earning goes lower, then it means for at least UC alumni, this field might be more promising(more money, more job opening, and possible eaiser internships and recruial). And thus under our H0, there shall be a upward shift in the numbers of studnet who choose this major if this major earnings is continously going higher(50%/median earning increase) / having a higher ceiling (75% earning increase) or higher bottom line (25% earning increase)

As noted above, the annual earnings here are in unit of adjusted US dollars, which means adjusting the salaries of previous years base on the ratio between the US current that year to 2024 year. The calculation behind it is a little bit complex, basic ideas is to find out how much money you need to spend in that year to buy for example an egg(buying power). You can use website like [CPI Inflation Calculator](https://www.bls.gov/data/inflation_calculator.htm) to do it. 

Several limitations of this dataset should be noted. This data is based on the Count of Number of Records, which many whom didn’t earn quite a good living would tend to not be on the record. Though we try our best to choose the represent majors, some rows have very few of UC alumni being recorded, for example, there is only 30 people are recorded in chemistry in 2025-2016 so their salary averages can swing wildly. Also, as for UC San Diego alumni only, the numbers of recorded are way too low beyond measurable, thus hereby we choose the UC wide alumni as representing UCSD alumni, which different schools may have different recognized majors, and this ego effect might also affect the annual salaries one may earned. For example, UCLA’s Visual art -media major is world-wide recognized, thus for when we compared UCSD’s enrollment (the first and previous dataset here) with UC alumni salaries (the second and this dataset here), we can’t reject there is a possibility here that the earning we used here is bit higher than the actual UCSD media major earning.
Please note that you can always keep adding more datasets in the future if these datasets you turn in for the checkpoint aren't sufficient.  The goal here is demonstrate that you can obtain and wrangle data.  You are not tied down to only use what you turn in right now.

#### Outlier & Suspicious Entry Flags

In [24]:
df_earn_flag = df_earn.copy()
rules = []
rules.append(("bad_year", ~df_earn_flag["year"].isin([2018, 2020, 2022, 2024])))
for c in ["earn_p25","earn_med","earn_p75"]:
    if c in df_earn_flag.columns:
        rules.append((f"neg_{c}", df_earn_flag[c] < 0))
        rules.append((f"too_big_{c}", df_earn_flag[c] > 1_000_000))
if set(["earn_p25","earn_med"]).issubset(df_earn_flag.columns):
    rules.append(("p25_gt_med", df_earn_flag["earn_p25"] > df_earn_flag["earn_med"]))
if set(["earn_med","earn_p75"]).issubset(df_earn_flag.columns):
    rules.append(("med_gt_p75", df_earn_flag["earn_med"] > df_earn_flag["earn_p75"]))

for c in ["earners_count","records_count"]:
    if c in df_earn_flag.columns:
        rules.append((f"neg_{c}", df_earn_flag[c] < 0))

def flag_iqr_outliers(s, k=1.5):
    s = s.dropna()
    if len(s) < 6:
        return pd.Series(False, index=s.index)
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - k * iqr, q3 + k * iqr
    return (s < lo) | (s > hi)

iqr_cols = [c for c in ["earn_p25","earn_med","earn_p75","earners_count"] if c in df_earn_flag.columns]
for col in iqr_cols:
    mask = (
        df_earn_flag.groupby("year", group_keys=False)[col]
        .apply(lambda s: flag_iqr_outliers(s, k=1.5))
    )
    df_earn_flag[f"out_{col}_iqr"] = False
    df_earn_flag.loc[mask.index, f"out_{col}_iqr"] = mask

df_earn_flag["suspicious_reasons"] = ""
for name, cond in rules:
    df_earn_flag.loc[cond.fillna(False), "suspicious_reasons"] += f"{name};"
for col in iqr_cols:
    flag_col = f"out_{col}_iqr"
    df_earn_flag.loc[df_earn_flag[flag_col] == True, "suspicious_reasons"] += f"{flag_col};"

df_earn_flag["is_suspicious"] = df_earn_flag["suspicious_reasons"].str.len() > 0
print("Suspicious rows (earnings):", df_earn_flag["is_suspicious"].sum())
display(df_earn_flag[df_earn_flag["is_suspicious"]].head(30))

Suspicious rows (earnings): 6


,year,major,earn_med,earn_p25,earn_p75,earners_count,records_count,out_earn_p25_iqr,out_earn_med_iqr,out_earn_p75_iqr,out_earners_count_iqr,suspicious_reasons,is_suspicious
2,2024,Computer Engineering,104844.0,76024.0,160505.0,156,156,False,False,True,True,out_earn_p75_iqr;out_earners_count_iqr;,True
4,2024,Electrical Engineering - BS,92359.0,79534.0,113449.0,172,172,False,False,False,True,out_earners_count_iqr;,True
17,2022,Computer Engineering,116188.0,78670.0,156585.0,127,127,False,False,True,False,out_earn_p75_iqr;,True
21,2022,Cognitive Science,59401.0,41686.0,81054.0,254,254,False,False,False,True,out_earners_count_iqr;,True
32,2020,Computer Engineering,121057.0,95474.0,160247.0,174,174,False,False,True,False,out_earn_p75_iqr;,True
47,2018,Computer Engineering,109959.0,88017.0,145813.0,174,174,False,False,True,False,out_earn_p75_iqr;,True


#### Tidy and Clean Data

In [26]:
import os, re
import numpy as np
import pandas as pd

RAW_DIR = "data/00-raw"
PROC_DIR = "data/02-processed"
os.makedirs(PROC_DIR, exist_ok=True)
earn_path = os.path.join(RAW_DIR, "UC alumni at work.csv")

df_earn_raw = pd.read_csv(
    earn_path,
    encoding="utf-8-sig",
    sep=",",
    engine="python",
)
print("Earnings raw shape:", df_earn_raw.shape)
display(df_earn_raw.head())

def standardize_major_name(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def to_numeric_money(x):
    """handles '120,548' or '$120,548' etc."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "":
        return np.nan
    s = s.replace("$", "").replace(",", "")
    try:
        return float(s)
    except:
        return np.nan

rename_map = {
    "Year": "year",
    "Major": "major",
    "Median Annual Earnings": "earn_med",
    "Percentile (25) of Annual Earnings": "earn_p25",
    "Percentile (75) of Annual Earnings": "earn_p75",
    "Earners Count": "earners_count",
    "Count of Number of Records": "records_count",
}
df_earn = df_earn_raw.rename(columns=rename_map).copy()

keep_cols = [c for c in rename_map.values() if c in df_earn.columns]
df_earn = df_earn[keep_cols].copy()
def normalize_year(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    m = re.match(r"^\s*(\d{4})\s*-\s*(\d{4})\s*$", s)
    if m:
        return int(m.group(2))
    try:
        return int(float(s))
    except:
        return np.nan

df_earn["year"] = df_earn["year"].apply(normalize_year).astype("Int64")
df_earn["major"] = df_earn["major"].apply(standardize_major_name)

for c in ["earn_med", "earn_p25", "earn_p75"]:
    if c in df_earn.columns:
        df_earn[c] = df_earn[c].apply(to_numeric_money)
for c in ["earners_count", "records_count"]:
    if c in df_earn.columns:
        df_earn[c] = pd.to_numeric(df_earn[c], errors="coerce")

YEARS = [2018, 2020, 2022, 2024]
df_earn = df_earn[df_earn["year"].isin(YEARS)].copy()

df_earn = df_earn.dropna(subset=["year", "major"]).copy()

print("Earnings cleaned shape:", df_earn.shape)
display(df_earn.head(10))

miss = df_earn.isna().mean().sort_values(ascending=False)
print("Earnings missingness (fraction):")
display(miss)

earn_out = os.path.join(PROC_DIR, "earnings_processed.csv")
df_earn.to_csv(earn_out, index=False, encoding="utf-8-sig")
print("Saved:", earn_out)

Earnings raw shape: (60, 7)


,Year,Major,Percentile (75) of Annual Earnings,Median Annual Earnings,Percentile (25) of Annual Earnings,Earners Count,Count of Number of Records
0,2024,History,49697,35933,21804,61,61
1,2024,Visual Arts - Media,49000,36057,20440,60,60
2,2024,Computer Engineering,160505,104844,76024,156,156
3,2024,Chemistry,71287,56269,48061,80,80
4,2024,Electrical Engineering - BS,113449,92359,79534,172,172


Earnings cleaned shape: (60, 7)


,year,major,earn_med,earn_p25,earn_p75,earners_count,records_count
0,2024,History,35933.0,21804.0,49697.0,61,61
1,2024,Visual Arts - Media,36057.0,20440.0,49000.0,60,60
2,2024,Computer Engineering,104844.0,76024.0,160505.0,156,156
3,2024,Chemistry,56269.0,48061.0,71287.0,80,80
4,2024,Electrical Engineering - BS,92359.0,79534.0,113449.0,172,172
5,2024,Bioengineering,61865.0,39206.0,83579.0,40,40
6,2024,Cognitive Science,44596.0,30211.0,67291.0,153,153
7,2024,Public Health,45419.0,29155.0,62040.0,62,62
8,2024,Applied Mathematics,56440.0,32556.0,72668.0,60,60
9,2024,Literature/Writing,38368.0,27740.0,50520.0,71,71


Earnings missingness (fraction):


year             0.0
major            0.0
earn_med         0.0
earn_p25         0.0
earn_p75         0.0
earners_count    0.0
records_count    0.0
dtype: float64

Saved: data/02-processed/earnings_processed.csv


## Ethics


Instructions: Keep the contents of this cell. For each item on the checklist
-  put an X there if you've considered the item
-  IF THE ITEM IS RELEVANT place a short paragraph after the checklist item discussing the issue.
  
Items on this checklist are meant to provoke discussion among good-faith actors who take their ethical responsibilities seriously. Your teams will document these discussions and decisions for posterity using this section.  You don't have to solve these problems, you just have to acknowledge any potential harm no matter how unlikely.

Here is a [list of real world examples](https://deon.drivendata.org/examples/) for each item in the checklist that can refer to.

[![Deon badge](https://img.shields.io/badge/ethics%20checklist-deon-brightgreen.svg?style=popout-square)](http://deon.drivendata.org/)

### A. Data Collection
 - [ ] **A.1 Informed consent**: If there are human subjects, have they given informed consent, where subjects affirmatively opt-in and have a clear understanding of the data uses to which they consent?
 - [X] **A.2 Collection bias**: Have we considered sources of bias that could be introduced during data collection and survey design and taken steps to mitigate those?  

            Admissions and earnings data may reflect structural inequalities, differences in applicant pools, and labor market demand. These systemic factors may influence observed associations.
 - [X] **A.3 Limit PII exposure**: Have we considered ways to minimize exposure of personally identifiable information (PII) for example through anonymization or not collecting information that isn't relevant for analysis?  

            The dataset contains only major-by-year aggregated statistics and no personally identifiable information.
 - [X] **A.4 Downstream bias mitigation**: Have we considered ways to enable testing downstream results for biased outcomes (e.g., collecting data on protected group status like race or gender)?  

            Because we analyze aggregated data, we cannot evaluate individual-level fairness. We explicitly state this limitation in our discussion.

### B. Data Storage
 - [X] **B.1 Data security**: Do we have a plan to protect and secure data (e.g., encryption at rest and in transit, access controls on internal users and third parties, access logs, and up-to-date software)?  

            Data is stored locally on password-protected devices and in a private GitHub repository accessible only to team members.
 - [ ] **B.2 Right to be forgotten**: Do we have a mechanism through which an individual can request their personal information be removed?  

            No individual-level data is collected, so this is not applicable.
 - [X] **B.3 Data retention plan**: Is there a schedule or plan to delete the data after it is no longer needed?  

            Data will be retained only for the duration of the course project.

### C. Analysis
 - [X] **C.1 Missing perspectives**: Have we sought to address blindspots in the analysis through engagement with relevant stakeholders (e.g., checking assumptions and discussing implications with affected communities and subject matter experts)?  

            Our analysis does not account for demographic factors, socioeconomic background, or non-monetary educational value.
 - [X] **C.2 Dataset bias**: Have we examined the data for possible sources of bias and taken steps to mitigate or address these biases (e.g., stereotype perpetuation, confirmation bias, imbalanced classes, or omitted confounding variables)?  

            Earnings may reflect broader economic trends, and admission rates may depend on applicant volume. These may confound interpretation.
 - [X] **C.3 Honest representation**: Are our visualizations, summary statistics, and reports designed to honestly represent the underlying data?  

            We report correlations and regression results transparently and avoid causal claims.
 - [X] **C.4 Privacy in analysis**: Have we ensured that data with PII are not used or displayed unless necessary for the analysis?  

            All results are based on aggregated statistics; no individual data is displayed.
 - [X] **C.5 Auditability**: Is the process of generating the analysis well documented and reproducible if we discover issues in the future?  

            All preprocessing and modeling steps are documented in the notebook to ensure reproducibility.
### D. Modeling
 - [X] **D.1 Proxy discrimination**: Have we ensured that the model does not rely on variables or proxies for variables that are unfairly discriminatory?  

            Although no sensitive attributes are used, selectivity and earnings may indirectly reflect systemic inequality.
 - [ ] **D.2 Fairness across groups**: Have we tested model results for fairness with respect to different affected groups (e.g., tested for disparate error rates)?
 - [X] **D.3 Metric selection**: Have we considered the effects of optimizing for our defined metrics and considered additional metrics?  

            We use multiple selectivity measures and regression models rather than relying on a single metric.
 - [X] **D.4 Explainability**: Can we explain in understandable terms a decision the model made in cases where a justification is needed?  

            We use interpretable linear regression models.
 - [X] **D.5 Communicate limitations**: Have we communicated the shortcomings, limitations, and biases of the model to relevant stakeholders in ways that can be generally understood?  

            We clearly state that correlation does not imply causation.

### E. Deployment
 - [X] **E.1 Monitoring and evaluation**: Do we have a clear plan to monitor the model and its impacts after it is deployed (e.g., performance monitoring, regular audit of sample predictions, human review of high-stakes decisions, reviewing downstream impacts of errors or low-confidence decisions, testing for concept drift)?  

            This is an academic analysis and not a deployed system.
 - [ ] **E.2 Redress**: Have we discussed with our organization a plan for response if users are harmed by the results (e.g., how does the data science team evaluate these cases and update analysis and models to prevent future harm)?
 - [ ] **E.3 Roll back**: Is there a way to turn off or roll back the model in production if necessary?
 - [X] **E.4 Unintended use**: Have we taken steps to identify and prevent unintended uses and abuse of the model and do we have a plan to monitor these once the model is deployed?  

            Findings could be misinterpreted as ranking majors solely by income; we caution against oversimplification.

## Team Expectations 

Below is our Team Expectations

## Communication

Primary Communication: WeChat will be our main communication platform. We will create a group chat for all project-related discussions. For important discussions, we may also schedule in-person meetings.

Response Time: All team members are expected to respond to messages within 24 hours. If there is an urgent issue (e.g., upcoming deadline or ready check before submission), members should respond as soon as possible.

Meetings: We will hold in-person meetings when necessary (especially before major deadlines). If someone cannot attend, they must inform the group in advance and review meeting notes afterward.

Respectful Communication: We will communicate respectfully and professionally. It is okay to disagree, but we will discuss issues constructively and focus on problem-solving rather than personal criticism.

## Task Completion & Responsibility

Each member is responsible for completing their assigned tasks on time.

If someone anticipates difficulty meeting a deadline, they must inform the group as soon as possible so we can adjust responsibilities.

We will use GitHub (Issues/Projects board) to track task progress and ensure transparency.

All members should contribute meaningfully to coding, analysis, writing, and presentation work.

## Addressing Problems

If a team member becomes unresponsive or does not complete tasks:

First, we will check in privately to understand the situation.

If the issue continues, we will discuss it as a group.

If the problem persists and affects project progress, we will inform the TA and/or professor.

## Commitment

All team members agree to:

Communicate clearly and respectfully.

Contribute fairly to the project.

Support each other throughout the quarter.

Work toward completing the project successfully and professionally.

## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them

| Meeting Date | Meeting Time | Completed Before Meeting | Discuss at Meeting |
|---|---|---|---|
| 1/20 | 1 PM | Read COGS 108 expectations; brainstorm UCSD-focused research ideas | Finalize research question: UCSD transfer selectivity and major-level earnings |
| 1/26 | 10 AM | Collect UCSD transfer admissions data and major-level earnings data | Review dataset structure; identify key variables (major, year, gpa_mid, admit_rate, earn_med, earn_p25, earn_p50, earn_p75) |
| 2/1 | 10 AM | Begin data cleaning; convert percentages and currency formats | Discuss construction of selectivity metrics (GPA midpoint, admission rate); check missing values |
| 2/16 | 6 PM | Complete full data cleaning and merge by major × year (due 2/16) | Confirm final cleaned dataset; verify merged variables and sample size |
| 2/23 | 12 PM | Complete EDA: trends in gpa_mid and admit_rate by major; earnings distributions | Interpret EDA results; refine hypotheses about selectivity–earnings relationship |
| 2/28 | 6 PM | Finalize EDA checkpoint | Plan statistical analysis: Pearson/Spearman correlations; OLS regressions (gpa_mid ~ earn_med, admit_rate ~ earn_med + year + C(major)) |
| 3/10 | 12 PM | Run regression models; include year and major fixed effects | Interpret coefficients; discuss limitations and potential confounders |
| 3/17 | 6 PM | Draft final results, discussion, and ethics section | Final review and edits before submission |
| 3/18 | Before 11:59 PM | NA | Submit Final Project & Group Surveys |
